## Инструмент веб-поиска

В этом ноутбуке мы используем инструмент веб-поиска для нахождения средней стоимости фитнес-центра в вашем городе.

Для начала установим необходимые библиотеки:

In [1]:
%pip install openai dotenv

Note: you may need to restart the kernel to use updated packages.


**ВНИМАНИЕ**: После установки библиотек рекомендуется перезапустить Kernel ноутбука.

И ещё полезная функция на будущее:

In [7]:
from IPython.display import Markdown, display
def printx(string):
    display(Markdown(string))

А также сделаем полезные импорты на будущее:

In [2]:
import os
import json

## Авторизация и создание клиента OpenAI

Для работы с языковыми моделями нам понадобится авторизоваться в Yandex Cloud. Для доступа к модели необходимы:
* идентификатор каталога `folder_id`
* API-ключ сервисного аккаунта `api_key`. Сервисный аккаунт должен иметь права на доступ к модели (рекомендуем `ai.editor`).

Мы предполагаем, что соответствующие значения хранятся в переменных окружения. Удобно использовать два подхода:
* Если вы разворачиваете код в Yandex Datasphere - используйте секреты проекта
* Если вы запускаете проект со своего компьютера - разместите значения ключей в файле `.env` (в корневом каталоге репозитория) и запустите следующую ячейку для их загрузки в переменные окружения

In [4]:
from dotenv import load_dotenv
load_dotenv()

True

In [5]:
folder_id = os.environ["folder_id"]
api_key = os.environ["api_key"]

Создадим клиент OpenAI и убедимся, что он работает:

In [8]:
from openai import OpenAI

model = f"gpt://{folder_id}/qwen3-235b-a22b-fp8/latest"

client = OpenAI(
    base_url="https://ai.api.cloud.yandex.net/v1",
    api_key=api_key,
    project=folder_id
)

res = client.responses.create(
    model = model,
    input = "Сколько в среднем стоит годовой абонемент на занятия в фитнес-клубе в Москве?"
)

printx(res.output_text)

Средняя стоимость годового абонемента в фитнес-клуб в Москве в 2024–2025 году составляет **от 60 000 до 150 000 рублей** в год, в зависимости от сети, уровня клуба, расположения и набора услуг.

Вот более детальная разбивка:

- **Бюджетные/доступные сети (например, «Спортмастер Фитнес», «Физкульт», «Джамп», «World Class Budget»)**  
  → от **60 000 до 90 000 руб./год**

- **Средний сегмент (например, «Арена Фитнес», «X-Fit», «Калининград Фитнес», «Здоровье»)**  
  → от **90 000 до 120 000 руб./год**

- **Премиум-клубы (например, World Class, Gold’s Gym, FitCurves, Equinox, Evviva)**  
  → от **120 000 до 250 000+ руб./год**, особенно в центре Москвы или с VIP-доступом

**Что влияет на цену:**
- Расположение (центр города дороже)
- Наличие бассейна, сауны, групповых занятий (пилатес, йога, кроссфит)
- Работа тренеров, индивидуальные программы
- Круглосуточный доступ
- Дополнительные услуги (массаж, косметология, детская комната)

**Совет:** Часто выгоднее покупать абонемент на 6–12 месяцев при открытии нового филиала — в это время действуют скидки до 30–50%.

Если рассматривать **среднюю цену по городу**, можно ориентироваться на **100 000 рублей в год** за стандартный абонемент с доступом ко всем зонам (тренажерный зал, групповые программы).

Однако такой ответ не является достаточно достоверным, поскольку
* Он основан на внутренней памяти языковой модели, и поэтому весьма вероятны галлюцинации
* Он основан на данных на момент обучения модели, поэтому не имеет доступа к последней информации.

Для получения более актуальных ответов на основе информации из интернет можем использовать инструмент поиска.

## Инструмент веб-поиска

Прелесть Responses API состоит в том, что он может осуществить предварительный поиск в интернет самостоятельно - вам лишь нужно указать соответствующий инструмент при вызове модели:

In [10]:
system_prompt = """
Ты - профессиональный фитнес-ассистент. 
Отвечай как энергичный молодой человек со спортивным задором.
В случае необходимости осуществляй поиск в интернет",
"""

res_ws = client.responses.create(
    model = model,
    instructions = system_prompt,
    tools = [ { "type": "web_search" } ],
    input = "Сколько в среднем стоит годовой абонемент на занятия в фитнес-клубе в Москве?"
)

printx(res_ws.output_text)

Йоу, привет! 💪 Только что пробежался по свежим данным — и вот тебе «горячая» инфа по ценам на годовые абонементы в московские фитнесы!

Средняя цена **в 2025 году** — от **14 000 до 35 000 рублей** в год, в зависимости от клуба, локации и уровня сервиса. А вот конкретные цифры:

- **Бюджетный сегмент** (от 8 800 ₽):  
  🔹 **Alex Fitness** — от 8 800 ₽  
  🔹 **СпортЛэнд** — от 13 900 ₽  
  🔹 **Зебра** — от 19 900 ₽  

- **Средний класс** (20–35 тыс. ₽):  
  🔹 **X-Fit** — от 23 000 ₽  
  🔹 **WeGym** — от 33 200 ₽  
  🔹 **World Class** — от ~34 700 ₽ (со скидкой)  
  🔹 **С.С.С.Р.** — от 35 900 ₽  

- **Премиум-уровень** (от 60 тыс. ₽):  
  🔹 **Dr. Loder** — от 62 000 ₽  

💡 **Лайфхак:**  
✅ Всегда спрашивай о скидках — их дают часто (до 30–50%)!  
✅ Ищи акции «в честь открытия» нового филиала — там цены сочные.  
✅ Гостевой визит — твой лучший друг! Проверь зал, тренажёры и атмосферу.

Хочешь — могу помочь подобрать клуб под твои цели и район! 💬 Жми в личку — и вперёд к новым результатам! 🏋️‍♂️🔥

Обратите внимание, что стиль ответа изменился - это из-за системного промпта. Но также мы добавили в системный промпт указание, что стоит искать информацию в интернет - это подскажет модели лишний раз использовать инструмент веб-поиска.

Также отметим, что длина объекта-ответа изменилась:

In [15]:
print(f"Ответ без поиска: {len(res.output)}")
print(f"Ответ с поиском: {len(res_ws.output)}")

Ответ без поиска: 1
Ответ с поиском: 2


Это произошло потому, что в ответе присутствует отдельная запись для вызова инструмента веб-поиска:

In [19]:
print(res_ws.output[0].action.query)
print(res_ws.output[0])

{"query": "средняя стоимость годового абонемента в фитнес-клуб в Москве", "lang": "ru"}
ResponseFunctionWebSearch(id='f98b0e4e-5192-441b-b10e-33aebd435529', action=ActionSearch(query='{"query": "средняя стоимость годового абонемента в фитнес-клуб в Москве", "lang": "ru"}', type='search', queries=None, sources=None, valid=True), status='completed', type='web_search_call', valid=True)


Тут мы можем видеть запрос, который LLM сформулировала для поискового инструмента. Этот запрос был отправлен в Yandex-поиск, и результаты переданы LLM для формирования финального ответа.

Мы также можем посмотреть на использованные для формирования ответа источники:

In [43]:
for x in res_ws.output[1].content[0].annotations:
    print(f"{x.title} - {x.url}")

Битва фитнес-клубов: где дешевле заниматься спортом в Москве - https://www.sravni.ru/text/bitva-fitnes-klubov-gde-deshevle-zanimatsja-sportom-v-moskve/
Тренажерные залы: цены от 73 руб. в Москве – Рейтинг спортзалов – Zoon.ru - https://zoon.ru/msk/fitness/price/trenazhernyj_zal/
годовой абонемент в фитнес - Авито | Объявления в Москве: купить вещь, выбрать исполнителя или работу, подобрать недвижимость и транспорт по низкой цене | Авито - https://www.avito.ru/moskva?q=%D0%B3%D0%BE%D0%B4%D0%BE%D0%B2%D0%BE%D0%B9+%D0%B0%D0%B1%D0%BE%D0%BD%D0%B5%D0%BC%D0%B5%D0%BD%D1%82+%D0%B2+%D1%84%D0%B8%D1%82%D0%BD%D0%B5%D1%81



Обратите внимание, что запрос к поиску формулируется в том числе с учётом истории переписки. Например, мы можем продолжить начатый диалог:

In [33]:
res_ws_2 = client.responses.create(
    model = model,
    previous_response_id = res_ws.id,
    tools = [ { "type": "web_search" } ],
    input = "А поищи самый дешевый?"
)

printx(res_ws_2.output_text)

Йоу, дружище! 🔥 Только что прошёлся по свежим данным — и вот тебе **самый бюджетный фитнес в Москве** на 2025 год:

---

### 💰 **Самые дешёвые годовые абонементы в Москве:**

#### 1. **Территория Фитнеса**  
- **Цена:** от **13 900 ₽/год**  
- Есть филиалы в Зеленограде и других районах  
- Входит: тренажёрный зал, групповые занятия, кардиозона  
- 💡 *Лайфхак:* Ищи акции — бывает скидка до 50% при открытии новых залов!

#### 2. **Alex Fitness**  
- **Цена:** от **8 800 ₽/год**  
- Бюджетная сеть с хорошим оборудованием  
- Работает по принципу «всё включено» — без доплат  
- Есть залы в Южном и Северном Бутово

#### 3. **Зебра Фитнес**  
- **Цена:** от **19 900 ₽/год**  
- Просторные залы, чисто, адекватные тренеры  
- Подходит для тех, кто хочет больше, чем просто тренажёры

#### 4. **Фитнес-Паллада**  
- **Цена:** около **9 800 ₽/год**  
- Маленький, но уютный зал рядом с метро Панфиловская  
- Отзывы — в основном позитивные: «чисто, тёплая атмосфера, нет очередей»

---

### 🚨 Где **ещё дешевле**?  
На **Авито** и в Telegram-группах часто продают **переоформленные абонементы** по суперценам:  
- Например, в **Golden (Медведково)** — годовой абонемент за **13 000 ₽** (вместо 16 500)  
- В **X-Fit** — от **5 900 ₽/мес**, но при оплате за год и с заморозкой — выходит дешевле  

---

### 🛎️ **Совет от тренера:**  
✅ Всегда **посещай пробную тренировку** — оцени чистоту, оборудование, атмосферу.  
✅ Спрашивай про **доплаты** — бывает, за полотенца, шкафчик или душ просят отдельно.  
✅ Ищи **сезонные скидки** — чаще всего они в январе и сентябре.

---

Хочешь — могу скинуть **ссылки на конкретные филиалы** или **подобрать клуб по твоему району и бюджету**. Просто скажи, где ты и сколько готов тратить! 💬💪

Посмотрим на соответствующий поисковый запрос:

In [35]:
if len(res_ws_2.output)>1:
    print(res_ws_2.output[0].action.query)
else:
    print("Инструмент поиска не был вызван")

{"query": "самый дешёвый годовой абонемент в фитнес-клуб в Москве 2025", "lang": "ru"}


## Кастомизация инструмента поиска

Как правило, нас интересует стоимость фитнеса в конкретном городе, где мы живём. Мы можем указать город в запросе, но существует и другая возможность - ограничить поиск только определённой территорией или доменом.

Инструмент веб-поиска допускает следующие параметры (подробно описаны в [документации](https://yandex.cloud/ru/docs/ai-studio/concepts/agents/tools/websearch)):

* **filters** - устанавливает ограничение на область поиска. Можно ограничивать поиск по списку доментов (**allowed_domains**), или по региону (**user_location**). Список возможных регионов доступен [здесь](https://yandex.cloud/ru/docs/search-api/reference/regions).
* **search_context_size** - позволяет указать размер контекста, получаемого LLM из поиска. Возможные значения: **low**, **medium** (используется по умолчанию) и **high**. Более подробный контекст ведёт к повышению потребления токенов, но и позволяет получить более правильный и детальный ответ.

Посмотрим, как эти параметры используются на практике:


In [45]:
response = client.responses.create(
    model=model,
    input="Сделай краткий обзор отзывов на World Class Fitness",
    tools=[
        {
            "type": "web_search",
            "filters": {
                "allowed_domains": [
                    "otzovik.com"
                ],
                "user_location": {
                    "region": "213", # Москва
                },
            },
            "search_context_size": "medium", # варианты: low | medium | high
        }
    ]
)
printx(response.output_text)

На основе анализа отзывов о сети фитнес-клубов **World Class** можно выделить следующие ключевые моменты:

### ✅ **Плюсы:**
- **Современное оборудование** — тренажёры, особенно Technogym, отмечаются как новое и исправное.
- **Чистота и комфорт** — многие отмечают чистые раздевалки, душевые, бассейн и использование белоснежных полотенец.
- **Бассейн и СПА-зоны** — наличие бассейна, джакузи, саун и хамама оценивается положительно.
- **Гибкость по времени** — клубы работают с раннего утра до позднего вечера (например, с 7:00 до 24:00).
- **Профессиональные тренеры** — в некоторых отзывах подчёркивается высокий уровень инструкторов и индивидуальный подход.
- **Широкая сеть** — возможность посещать разные клубы сети в зависимости от локации.

---

### ❌ **Минусы:**
- **Высокая цена** — абонементы стоят дорого (от 90 000 руб./год и выше), что не всегда оправдано качеством сервиса.
- **Проблемы с сервисом** — частые жалобы на:
  - **Отказ в выдаче справок** для налогового вычета (ожидание до 3–6 месяцев).
  - **Игнорирование клиентов** со стороны администрации.
  - **Грубость и нехватку вежливости** у персонала.
- **Переполненность** — вечером и в «час пик» (18:00–21:00) залы переполнены, сложно найти свободный тренажёр.
- **Отсутствие парковки** — неудобство для тех, кто приезжает на машине.
- **Некомфортные условия в отдельных филиалах** — плохая вентиляция, запахи, грязь, сломанные тренажёры.
- **Ограничения для детей** — нехватка занятий для подростков, отсутствие раздевалок для переодевания с детьми.

---

### ⚠️ **Серьёзные проблемы:**
- **Инциденты с безопасностью** — в одном из отзывов описан случай смерти посетителя в зале, при этом персонал не оказал помощь.
- **Юридические претензии** — несколько отзывов упоминают судебные иски к клубу из-за нарушений договоров, отказа в возврате денег, мошенничества.

---

### 📊 **Общая оценка:**
- На сайте Otzovik.com **260 отзывов**, из них **только 28% рекомендуют** сеть.
- Средняя оценка — **2.3 из 5**, что указывает на **преимущественно негативное восприятие**.
- Хотя есть довольные клиенты, особенно отмечающие комфорт и качество тренировок, **большинство разочарованы сервисом, ценами и отношением к клиентам**.

---

### 💡 **Вывод:**
**World Class** — это фитнес-клуб премиум-сегмента с хорошей инфраструктурой, но **сервис и клиентоориентированность оставляют желать лучшего**. Если для вас важны **оборудование и бассейн**, и вы готовы заниматься в не пиковые часы — можно рассмотреть. Однако будьте готовы к **дороговизне, бюрократии и возможным конфликтам с администрацией**.

## Выводы

К вызову LLM достаточно легко добавить инструмент поиска, и все соответствующие сложности по вызову инструмента и обработке результата Yandex Cloud берёт на себя. Однако, следует помнить, что инструмент поиска [тарифицируется](https://yandex.cloud/ru/docs/ai-studio/pricing) отдельно, и это увеличивает стоимость запросов. Также, LLM сама решает, когда следует вызывать инструмент поиска, поэтому стоит указать какие-то правила вызова поиска в системном промпте и/или в самом запросе. 